# Youtube Video Title Generator
> The purpose of this project is to train a sequence model(LSTM) at Word-Level to suggest a title for youtube video

In [148]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

import random
from numpy.random import seed
#set_random_seed(2)
seed(1)


# NLP libraries
import nltk
import stanfordnlp
from sklearn.pipeline import Pipeline
from nltk.tokenize import RegexpTokenizer
import regex as re


# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 




import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('ggplot')



In [149]:
sa_trending = pd.read_csv('saudi_youtube_trending_videos.csv',index_col=0)



## simple NLP 
>
- Here I used stanfordnlp library 


In [150]:
corpus = sa_trending.video_name

In [151]:
nlp = stanfordnlp.Pipeline(processors='tokenize,mwt,pos,lemma,depparse', lang="ar")  # Full pipeline


Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/bodor/stanfordnlp_resources/ar_padt_models/ar_padt_tokenizer.pt', 'lang': 'ar', 'shorthand': 'ar_padt', 'mode': 'predict'}
---
Loading: mwt
With settings: 
{'model_path': '/Users/bodor/stanfordnlp_resources/ar_padt_models/ar_padt_mwt_expander.pt', 'lang': 'ar', 'shorthand': 'ar_padt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
---
Loading: pos
With settings: 
{'model_path': '/Users/bodor/stanfordnlp_resources/ar_padt_models/ar_padt_tagger.pt', 'pretrain_path': '/Users/bodor/stanfordnlp_resources/ar_padt_models/ar_padt.pretrain.pt', 'lang': 'ar', 'shorthand': 'ar_padt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/bodor/stanfordnlp_resources/ar_padt_models/ar_padt_lemmatizer.pt', 'lang': 'ar', 'shorthand': 'ar_padt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a 

In [152]:
# extraction arabic text only and remove any nun arabic letter
def extract_arabic_text(text):
    return re.sub("[^\u0600-\u06FF]"," ",text)

In [153]:

ar_corpus = []
for i in corpus:
    ar_corpus.append(extract_arabic_text(i))

In [154]:
ar_corpus[:10]

['أغرب المنتجات اللي ممكن تشتريها من الإنترنت        ',
 'مقلب إبلاغ الشرطة في الشارع           ',
 'حفلة ميلاد ريما عشان الخساير لكن    ',
 'فيديو بائعة العسل السعودية يثير ضجة على وسائل التواصل',
 'تحدي التشويت من نص الملعب       تحديات فارس',
 'كاج   واحد و عشرين                                ',
 'ردة فعلنا على فيديو حنون واهي صغيرة   عائلة عدنان',
 'لعبة غريبة  ',
 'يوم في محل الألعاب ',
 'تحدي كشف الكذب   عشت قصة حب من قبل؟',
 'الهدية اللي كانت تنتظرها  ',
 'البث المباشر للحفل الختامي   تحدي القراءة العربي     ',
 'نجمة مواقع التواصل الاجتماعي الطفلة رنيم محمد في ضيافة ساعة شباب',
 '                                                                              ',
 'دخلت مباراة الهلال متنكر وهذا اللي صار      ',
 'حكايا باب الحارة    أبو بشير يعتذر من أبو عصام ويطلب إيد جميلة   تاج حيدر  و عباس النوري',
 'اشخاص مسوين اشياء مجنونه   راح تعرق بسببهم     ',
 'وش يصير لو كفر دعس على مسامير ؟   ناز رياكشن',
 'أقوى تحديات ضد صبري     صدمني بمستواه  ',
 '                                  

In [155]:
def extract_lemma_token(doc):
    tokens = []
    lemmas = []
    for sent in doc.sentences:
        for wrd in sent.words:
            #extract text and lemma
            tokens.append(wrd.text)
            lemmas.append(wrd.lemma)
            #return a dataframe
    return " ".join(tokens)," ".join(lemmas)

In [156]:
def remove_stop_words(doc):
    arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
    words  = doc.split()
    return " ".join([w for w in words if w not in arb_stopwords])

In [157]:
tokens = []
lemmas = []
filt_stop_words = []
for i in ar_corpus:
    try:
        doc = nlp(i)
        t ,l =  extract_lemma_token(doc)
        tokens.append(t)
        lemmas.append(l)
        
        filt_stop_words.append(remove_stop_words(i,arb_stopwords))
    except:
        pass
    

    
    

In [159]:
lemmas[:10]

['أَغرَب مُنتَج لَيّ مُمكِن اِشتَرَى هُوَ مِن إِنتِرنِت',
 'مقلب إِبلَاغ شُرطَة فِي شَارِع',
 'حَفلَة ميلاد ريما عشان خَسَارَة لكن',
 'فِيدِيُو بَائِعَة عَسَل سَعُودِيّ أَثَار ضَجَّة عَلَى وَسِيلَة تَوَاصُل',
 'تَحَدِّي تَشوِيت مِن نَصّ مَلعَب تَحَدِّي فارس',
 'كاج وَاحِد وَ عِشرُون',
 'رَدَّة فَعَل عَلَى فِيدِيُو حنون وَ أَهَىّ صَغِير عَائِلَة عدنان',
 'لُعبَة غَرِيبَة',
 'يَوم فِي مَحَلّ لَعب',
 'تَحَدِّي كَشف كِذب عَشَى قِصَّة حُبّ مِن قَبلَ ؟',
 'هَدِيَّة لَيّ كَان تنتظرها',
 'بَثّ مُبَاشِر لِ حَفَل خِتَامِيّ تَحَدِّي قِرَاءَة عَرَبِيّ',
 'نَجمَة مَوقِع تَوَاصُل اِجتِمَاعِيّ طِفلَة رَنِيم محمد فِي ضيافة سَاعَة شَبّ',
 'دَخَل مُبَارَاة هِلَال متنكر وَ هٰذَا لَيّ صَار',
 'حَكِيَّة بَاب حَارَة أبو ب شَير اِعتَذَر مِن أبو عصام وَ طَلَب أَيَّد جَمِيل تاج حيدر وَ عباس النوري',
 'شَخص مسوين شَيء مَجنُون هُوَ رَاح عَرَّق بسببهم',
 'وَشّ أَصَار لَو كفر دَعس عَلَى مسامير ؟ نَاز رِيَاكِشن',
 'أَقوَى تَحَدِّي ضِدَّ صبري صدمني بمستواه',
 'الدحيح الشاويش بُطُوط',
 'هٰذَا هُوَ أَفضَل مُدَرِّب فِي

In [ ]:
filt_stop_words[:10]

## LSTM

> 
   - step 1 : use ar_corpus,which only contain arabia letters
   - step 2: Tokenization ,word embedding and  possible sequence of words
   - step 3 : Shuffle and split training set
   - step 4: create and train model
   - step 5: predecting
   

In [160]:
tokenizer = Tokenizer()

def get_sequence(corpus):
    
    #tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    # get possible sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

in_sequences, vocab_total = get_sequence(tokens)

In [161]:
len(in_sequences)

1658

In [162]:
vocab_total

1130

In [163]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len


#get predictors(X), label (y), and maximum length of words(input_len)
X, Y, max_sequence_len = generate_padded_sequences(in_sequences)

In [164]:
input_len = max_sequence_len - 1

In [165]:
model = Sequential()
# input layer
model.add(Embedding(vocab_total, 5, input_length=input_len))

# 2 LSTM layers
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))

# connected layer
model.add(Dense(100, activation='relu'))

# output layer
model.add(Dense(vocab_total, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 22, 5)             5650      
_________________________________________________________________
lstm_5 (LSTM)                (None, 22, 100)           42400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 1130)              114130    
Total params: 252,680
Trainable params: 252,680
Non-trainable params: 0
_________________________________________________________________
None


In [166]:
# fitting and training

model.fit(X, y, batch_size=128, epochs=100)


Epoch 1/100
1658/1658 [==============================] - 4s 3ms/step - loss: 7.0238 - accuracy: 0.0428
Epoch 2/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.8034 - accuracy: 0.0476
Epoch 3/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.5333 - accuracy: 0.0476
Epoch 4/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.4601 - accuracy: 0.0476
Epoch 5/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.4285 - accuracy: 0.0476
Epoch 6/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.4178 - accuracy: 0.0476
Epoch 7/100
1658/1658 [==============================] - 2s 975us/step - loss: 6.4156 - accuracy: 0.0476
Epoch 8/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.4099 - accuracy: 0.0476
Epoch 9/100
1658/1658 [==============================] - 2s 1ms/step - loss: 6.4100 - accuracy: 0.0476
Epoch 10/100
1658/1658 [==============================] - 2s 1ms/step -

### Testing(Generating)




In [167]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [168]:
generate_text("تحدي", 3, model, max_sequence_len)

'تحدي الملعب الصابوني حماس'

In [169]:
generate_text("فديو", 4, model, max_sequence_len)

'فديو الكويتية اعمتني العاصفة الثلجية'

In [170]:
generate_text("حفل", 2, model, max_sequence_len)

'حفل بحر رياكشن'

In [171]:
generate_text("قصة",3, model, max_sequence_len)

'قصة اعمتني العاصفة الثلجية'